# Update torus_points float.h from CSV
This notebook reads `main/plugins/rgb/resources/LED torus points float.csv`, extracts the per-slice XYZ points, and writes an updated header `main/plugins/rgb/resources/torus_points float.h` with the new `TORUS_POINTS_COUNT` and `TORUS_POINTS_PER_SLICE`. It will also create a backup of the existing header before overwriting.

In [4]:
# Ensure required packages
import sys, subprocess
def ensure(pkg, import_name=None):
    try:
        __import__(import_name or pkg)
    except ImportError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg])

ensure('pandas')
ensure('numpy')

import pandas as pd
import numpy as np
import re
from pathlib import Path
import shutil

# Paths (repo-root relative)
csv_path = Path('LED torus points float.csv')
hdr_path = Path('torus_points float.h')
backup_path = hdr_path.with_suffix('.h.bak')

print('CSV:', csv_path)
print('Header:', hdr_path)

CSV: LED torus points float.csv
Header: torus_points float.h


In [5]:
# Read CSV and organize per-slice points
df = pd.read_csv(csv_path)
cols = list(df.columns)
# Expect columns like X0,Y0,Z0,X1,Y1,Z1,...
pattern = re.compile(r'^\s*([XYZxyz])(\d+)\s*$')
slices = {}
for col in cols:
    m = pattern.match(col)
    if not m:
        raise ValueError(f'Unexpected column name: {col}')
    axis = m.group(1).upper()
    idx = int(m.group(2))
    slices.setdefault(idx, {})[axis] = df[col].values

n_slices = max(slices.keys()) + 1
# build per-slice list of (x,y,z) using rows as point index
per_slice_points = []
for i in range(n_slices):
    axes = slices.get(i)
    if axes is None or not all(k in axes for k in ('X','Y','Z')):
        raise ValueError(f'Slice {i} missing axis columns')
    pts = list(zip(axes['X'].astype(float), axes['Y'].astype(float), axes['Z'].astype(float)))
    per_slice_points.append(pts)

# Validate counts
counts = [len(pts) for pts in per_slice_points]
if len(set(counts)) != 1:
    raise ValueError(f'Inconsistent points-per-slice: {counts[:8]}...')
n_per_slice = counts[0]
print(f'slices={n_slices}, points_per_slice={n_per_slice}, total_points={n_slices * n_per_slice}')

slices=64, points_per_slice=241, total_points=15424


In [6]:
# Desired update: set per-slice to 241 (user requested) and rewrite header accordingly.
desired_per_slice = 241
if n_per_slice != desired_per_slice:
    print('WARNING: CSV contains', n_per_slice, 'points per slice, but requested', desired_per_slice)
    # We'll still write whatever the CSV contains, but update will reflect actual CSV size unless user overrides.

n_per_slice_to_write = n_per_slice  # use actual CSV value
total = n_slices * n_per_slice_to_write

# Read existing header to capture scale comment if present
existing = ''
if hdr_path.exists():
    existing = hdr_path.read_text(encoding='utf-8')
    # try to capture Scale applied line
    m = re.search(r'Scale applied:\s*(.*)', existing)
    scale_line = m.group(1).strip() if m else ''
else:
    scale_line = ''

# Build header content
def fmt(v):
    return f'{v:.6f}f'

lines = []
lines.append('/* Auto-generated torus point LUT')
lines.append(f'   Count: {total}')
lines.append(f'   Slices: {n_slices}')
lines.append(f'   Points per slice: {n_per_slice_to_write}')
if scale_line:
    lines.append(f'   Scale applied: {scale_line}')
else:
    lines.append('   Scale applied: none')
lines.append('   Stored as: float (32-bit)')
lines.append('*/')
lines.append('')
lines.append('#ifndef TORUS_POINTS_H')
lines.append('#define TORUS_POINTS_H')
lines.append('')
lines.append('#include <stdint.h>')
lines.append('#include <stddef.h>')
lines.append('')
lines.append(f'#define TORUS_POINTS_COUNT {total}')
lines.append(f'#define TORUS_POINTS_SLICES {n_slices}')
lines.append(f'#define TORUS_POINTS_PER_SLICE {n_per_slice_to_write}')
lines.append('')
lines.append(f'static const float TORUS_POINTS[{n_slices}][{n_per_slice_to_write}][3] = {{')

# Write slices
for si, pts in enumerate(per_slice_points):
    lines.append('  {')
    for (x,y,z) in pts:
        lines.append(f'    {{ {x:.6f}f, {y:.6f}f, {z:.6f}f }},')
    lines.append('  },')

lines.append('};')
lines.append('')
lines.append('#endif // TORUS_POINTS_H')

header_text = '\n'.join(lines)
# Backup existing header
if hdr_path.exists():
    shutil.copy2(hdr_path, backup_path)
    print('Backup written to', backup_path)
# Write new header
hdr_path.write_text(header_text, encoding='utf-8')
print('Wrote header:', hdr_path)
print('New TORUS_POINTS_COUNT =', total)
print('New TORUS_POINTS_PER_SLICE =', n_per_slice_to_write)

Backup written to torus_points float.h.bak
Wrote header: torus_points float.h
New TORUS_POINTS_COUNT = 15424
New TORUS_POINTS_PER_SLICE = 241


**Run instructions**: Open this notebook, run every cell. The notebook will back up the existing header to `torus_points float.h.bak` and overwrite `torus_points float.h` with the CSV-derived data. If you want to force 241 points per slice when CSV differs, modify the `n_per_slice_to_write` variable in the cell before writing.